## 01. 라이브러리 로드

In [1]:
import pandas as pd

## 02. 데이터 로드

In [2]:
df = pd.read_excel("data/전처리후데이터셋.xlsx",dtype={'거래소코드':str})

df['회계년도'] = df['회계년도'].str.split('/').str[0].astype(int)
df = df.sort_values(["회사명","회계년도"],ascending=True)

## 03. 종속변수 라벨링

#### 이자보상배율 / 영업활동현금흐름 조건 라벨링

In [3]:
# 이자보상배율 조건 계산 (2년 연속 0 미만)
df['이자보상배율_1미만'] = (df['이자보상배율(이자비용)'] < 1).astype(int)
df['이자보상배율_롤링합'] = df.groupby('회사명')['이자보상배율_1미만'].rolling(window=2).sum().reset_index(0, drop=True)
df['이자보상배율라벨'] = 0
df.loc[df['이자보상배율_롤링합'] == 2.0, '이자보상배율라벨'] = 1
df = df.drop(columns=['이자보상배율_1미만', '이자보상배율_롤링합'])

# 영업활동현금흐름 조건 계산 (2년 연속 0 미만)
df['영업현금흐름_0미만'] = (df['영업활동으로 인한 현금흐름(*)(천원)'] < 0).astype(int)
df['영업현금흐름_롤링합'] = df.groupby('회사명')['영업현금흐름_0미만'].rolling(window=2).sum().reset_index(0, drop=True)
df['영업현금흐름라벨'] = 0
df.loc[df['영업현금흐름_롤링합'] == 2.0, '영업현금흐름라벨'] = 1
df = df.drop(columns=['영업현금흐름_0미만', '영업현금흐름_롤링합'])

#### 이자보상배율 / 영업활동현금흐름 조건 두개 합치기 (4진분류) - create_four_class_label 함수 이용
- target_class_next_year : 다음해의 라벨예측을 위한 조정 (2013년 재무데이터 -> 2014년 라벨 예측)

In [4]:
def create_four_class_label(interest_label, cashflow_label):
    """
    두 개의 이진 라벨을 4진분류로 변환
    
    매핑 규칙:
    - 이자보상배율라벨=0, 영업현금흐름라벨=0 → 0
    - 이자보상배율라벨=1, 영업현금흐름라벨=0 → 1  
    - 이자보상배율라벨=0, 영업현금흐름라벨=1 → 2
    - 이자보상배율라벨=1, 영업현금흐름라벨=1 → 3
    """
    return interest_label * 2 + cashflow_label

df['target_class'] = df.apply(lambda row: create_four_class_label(
    row['이자보상배율라벨'], 
    row['영업현금흐름라벨']
), axis=1)

df['target_class_next_year'] = df.groupby("회사명")['target_class'].shift(-1)
df.dropna(inplace=True)
df['target_class_next_year'] = df['target_class_next_year'].astype(int)

#### 이자보상배율라벨, 영업현금흐름라벨 제거 및 컬럼명 재정의

In [5]:
df.drop(columns=["이자보상배율라벨","영업현금흐름라벨","target_class"], inplace=True)
df.rename(columns={"target_class_next_year" : "target_class"}, inplace=True)

## 04. 라벨 별 분포 확인
- 0 : 정상기업 & 현금흐름 양호
- 1 : 정상기업 & 현금흐름 불량
- 2 : 한계기업 & 현금흐름 양호
- 3 : 한계기업 & 현금흐름 불량

In [6]:
print("\n--- 4진 분류 타겟 분포 ---")
print(df['target_class'].value_counts().sort_index())
print("\n--- 타겟별 비율 ---")
print(df['target_class'].value_counts(normalize=True).sort_index())


class_labels = {
    0: '정상기업 & 현금흐름 양호',
    1: '정상기업 & 현금흐름 불량',
    2: '한계기업 & 현금흐름 양호',
    3: '한계기업 & 현금흐름 불량'
}


--- 4진 분류 타겟 분포 ---
target_class
0    13375
1      747
2     2212
3     1988
Name: count, dtype: int64

--- 타겟별 비율 ---
target_class
0    0.729997
1    0.040771
2    0.120729
3    0.108503
Name: proportion, dtype: float64


## 05. 데이터 저장

In [23]:
df.to_excel("data/label_dataset.xlsx",index=False)